# Importing the Libraries


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


###### Data retrieved from City of Surrey Open Data 
https://data.surrey.ca/dataset/restaurants

In [2]:
df = pd.read_csv(r'D:\restaurants.csv')

In [3]:
df.head()

,TRACKINGNUMBER,NAME,PHYSICALADDRESS,PHYSICALCITY,FACTYPE,LATITUDE,LONGITUDE
0,SWOD-APSP3X,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,SDFO-88ESVK,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,NDAA-8RNNVR,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,SYOG-5M5942,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,GEDS-6KHW57,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [6]:
pwd

'C:\\Users\\srira'

# Data Wrangling

In [4]:
df.drop('TRACKINGNUMBER',axis=1,inplace = True)

In [5]:
df.head()

,NAME,PHYSICALADDRESS,PHYSICALCITY,FACTYPE,LATITUDE,LONGITUDE
0,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [8]:
df.rename(columns={'PHYSICALADDRESS':'address','PHYSICALCITY':'city','NAME':'name'},inplace = True)

In [9]:
df.head()

,name,address,city,FACTYPE,LATITUDE,LONGITUDE
0,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [10]:
df.drop('FACTYPE',axis=1,inplace = True)

In [21]:
df= df.head(50)

In [22]:
df.dtypes

name          object
address       object
city          object
LATITUDE     float64
LONGITUDE    float64
dtype: object

In [13]:
address = 'Surrey, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\srira\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 49.1913033, -122.8491439.


In [16]:
map_surrey = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(df['LATITUDE'], df['LONGITUDE'], df['NAME']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_surrey)  
    
map_surrey

##### Getting the foursquare credentials

In [83]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


# Exploratory Data Analysis

##### Exploring the first restaurant and the nearby venues

In [15]:
# df.loc[0,'ADDRESS']
latitude = df.loc[1,'LATITUDE']
longitude = df.loc[1,'LONGITUDE']

##### Setting the foursquare request URL

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ONAZS2PTG1GXEGVC5IFRCK4H3E5LLKODWWCXIBCQGI1VZBHC&client_secret=0XJQQBLBWNGYGWKLTNHOWDBWAAJMY53XOXOVF0WX3WRNCQ0R&v=20180605&ll=49.10544931,-122.70131471&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
# results

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

nearby_venues1.head()

,name,categories,lat,lng
0,The Clayton Public House,Pub,49.104655,-122.701060
1,Sunrise Golf Centre,Golf Course,49.105024,-122.701071
2,Mavericks Taphouse And Grill,Pub,49.104966,-122.701028


##### The above cell shows that there are three popular venues near the first restaurant ' 104 Sushi and Co' . There are two pubs and a golf course. Probably not a good idea to open a restaurant near noisy unpleasant pubs

##### Now lets get the nearby venues for all the restaurants from our Dataset

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
surrey_venues = getNearbyVenues(names=df['name'],
                                   latitudes=df['LATITUDE'],
                                   longitudes=df['LONGITUDE']
                                  )

104 Sushi & Co.
5 Star Catering
555 Pizza Ltd
7-Eleven #26365
7-Eleven #26517
7-Eleven #34339
7-Eleven #34501
7-Eleven #34633
7-Eleven #34907
7-Eleven #35770
7-Eleven #36022H
7-Eleven #37876H
7-Eleven #38206
7-Eleven (Canada Store #37206H)
7-Eleven (Canada Store #37865H)
7-Eleven (Canada Store #37866H)
7-Eleven Canada Store #36676
7-Eleven Store # 37868
7-ElevenStore and Gas Bar (Canada Store #37875H)
North Surrey Secondary School Cafeteria
Northview Golf & Country Club (Canal Kiosk)
Northview Golf & Country Club (Kitchen/Duffey's Sports Grill)
Northview Golf & Country Club (Palmer Dining Room)
Northview Golf & Country Club (Ridge Kiosk)
Notti Biscotti
Nutri-Dough Pizza
O My Tea Cafe
Occasions at the Pond
Ocean Park Donair
Ocean Park Pizza
Ocean Park Pizza & Village Pub
Ocean Park Pizza (72nd Ave)
Oh So Sweet Sugar Boutique
Ok Hee Bae Food Co
Okonomiyaki Pon
Old Surrey Restaurant
Omega Pizza & Wings
Omega Pizza & Wings (152nd St)
On Yuen Restaurant
One More Thai Restaurant
One Up Cafe


In [51]:
surrey_venues.head(10)

,name,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,104 Sushi & Co.,49.192059,-122.756256,Fraser Glen Pizza,49.191401,-122.756039,Pizza Place
1,104 Sushi & Co.,49.192059,-122.756256,Flying Cup,49.191845,-122.756420,Coffee Shop
2,104 Sushi & Co.,49.192059,-122.756256,104 Sushi & Co,49.192245,-122.756318,Sushi Restaurant
3,104 Sushi & Co.,49.192059,-122.756256,777 Bubble Tea,49.191919,-122.756211,Tea Room
4,5 Star Catering,49.105449,-122.701315,The Clayton Public House,49.104655,-122.701060,Pub
5,5 Star Catering,49.105449,-122.701315,Sunrise Golf Centre,49.105024,-122.701071,Golf Course
6,5 Star Catering,49.105449,-122.701315,Mavericks Taphouse And Grill,49.104966,-122.701028,Pub
7,555 Pizza Ltd,49.170274,-122.889988,Thai By Thai,49.169821,-122.889959,Thai Restaurant
8,7-Eleven #26365,49.148189,-122.890221,The Keg Steakhouse + Bar,49.147487,-122.890022,Steakhouse
9,7-Eleven #26365,49.148189,-122.890221,Cactus Club Cafe,49.146924,-122.890932,American Restaurant


##### Here is where it gets interesting. For a restaurant, the nearby venues retrieved from Foursquare shoul return the same restaurant as a spot as well. Well, only if the restaurant is popular or  preferred. As you can see above,  for some restaurants like the '5 Star Catering', the nearby venues doesnt include them in the results. This could be because they might be just a 'take away' restaurant with no dine in facilities. In such cases, its a potential good spot without having to worry about competition.

In [33]:
surrey_venues.shape

(227, 7)

#####  How many popular venues are around each restaurant?

In [52]:
surrey_venues.groupby('name').count()

,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
name,,,,,,
104 Sushi & Co.,4,4,4,4,4,4
5 Star Catering,3,3,3,3,3,3
555 Pizza Ltd,1,1,1,1,1,1
7-Eleven #26365,8,8,8,8,8,8
7-Eleven #26517,3,3,3,3,3,3
7-Eleven #34339,1,1,1,1,1,1
7-Eleven #34501,2,2,2,2,2,2
7-Eleven #34633,10,10,10,10,10,10
7-Eleven #34907,4,4,4,4,4,4


In [35]:
print('There are {} uniques categories.'.format(len(surrey_venues['Venue Category'].unique())))

There are 67 uniques categories.


##### One hot Encoding the Venues

In [53]:
surrey_onehot = pd.get_dummies(surrey_venues[['Venue Category']], prefix="", prefix_sep="")

# # add restaurant column back to dataframe
surrey_onehot['name'] = surrey_venues['name'] 

# # move restaurantcolumn to the first column
fixed_columns = [surrey_onehot.columns[-1]] + list(surrey_onehot.columns[:-1])
surrey_onehot = surrey_onehot[fixed_columns]

surrey_onehot.head(10)

,name,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bank,Breakfast Spot,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Costume Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mexican Restaurant,Mobile Phone Shop,Music Store,Optical Shop,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Pet Store,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Vietnamese Restaurant,Women's Store
0,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5 Star Catering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5 Star Catering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,5 Star Catering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,555 Pizza Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,7-Eleven #26365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,7-Eleven #26365,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Grouping the results by the restaurants

In [54]:
surrey_grouped = surrey_onehot.groupby('name').mean().reset_index()

In [55]:
surrey_grouped.head()

,name,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bank,Breakfast Spot,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Costume Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mexican Restaurant,Mobile Phone Shop,Music Store,Optical Shop,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Pet Store,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Vietnamese Restaurant,Women's Store
0,104 Sushi & Co.,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.25,0.0,0.25,0.0,0.0,0.0
1,5 Star Catering,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,555 Pizza Ltd,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.00,1.0,0.0,0.0
3,7-Eleven #26365,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.125000,0.0,0.000000,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,7-Eleven #26517,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.0,0.0


In [56]:
num_top_venues = 5

for i in surrey_grouped['name']:
    print("----"+i+"----")
    temp = surrey_grouped[surrey_grouped['name'] == i].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----104 Sushi & Co.----
                venue  freq
0         Pizza Place  0.25
1            Tea Room  0.25
2    Sushi Restaurant  0.25
3         Coffee Shop  0.25
4  Mexican Restaurant  0.00


----5 Star Catering----
                  venue  freq
0                   Pub  0.67
1           Golf Course  0.33
2   American Restaurant  0.00
3  Pakistani Restaurant  0.00
4     Other Repair Shop  0.00


----555 Pizza Ltd----
                           venue  freq
0                Thai Restaurant   1.0
1            American Restaurant   0.0
2             Mexican Restaurant   0.0
3  Paper / Office Supplies Store   0.0
4           Pakistani Restaurant   0.0


----7-Eleven #26365----
                 venue  freq
0  American Restaurant  0.12
1          Coffee Shop  0.12
2          Gas Station  0.12
3    Indian Restaurant  0.12
4    Other Repair Shop  0.12


----7-Eleven #26517----
                 venue  freq
0          Flea Market  0.33
1          Gas Station  0.33
2    Convenience Store  0.33
3 

In [58]:
surrey_grouped.shape

(49, 68)

lets put that into a pandas dataframe

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Sorting the venues 

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
surrey_venues_sorted = pd.DataFrame(columns=columns)
surrey_venues_sorted['name'] = surrey_grouped['name']

for ind in np.arange(surrey_grouped.shape[0]):
    surrey_venues_sorted.iloc[ind, 1:] = return_most_common_venues(surrey_grouped.iloc[ind, :], num_top_venues)

surrey_venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,104 Sushi & Co.,Tea Room,Sushi Restaurant,Pizza Place,Coffee Shop,Women's Store,Fried Chicken Joint,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant
1,5 Star Catering,Pub,Golf Course,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop
2,555 Pizza Ltd,Thai Restaurant,Women's Store,Gift Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course
3,7-Eleven #26365,American Restaurant,Steakhouse,Greek Restaurant,Gas Station,Other Repair Shop,Convenience Store,Coffee Shop,Indian Restaurant,Bus Stop,Fish & Chips Shop
4,7-Eleven #26517,Gas Station,Convenience Store,Flea Market,Women's Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant


# Model Development

##### Using K Means Clustering to cluster an segment the community based on the restaurants and the popular spots returned by foursquare

In [62]:
kclusters = 5

surrey_grouped_clustering = surrey_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(surrey_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([0, 0, 4, 0, 1, 3, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 3, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0])

In [77]:
# surrey_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# surrey_venues_sorted.rename(columns={'RESTAURANT NAME':'rest'},inplace=True)
# df.rename(columns={'NAME':'rest'},inplace=True)
surrey_merged = df
surrey_merged = surrey_merged.join(surrey_venues_sorted.set_index('name'), on='name')

surrey_merged.head() 
# surrey_venues_sorted.head()
# df.head()

,name,address,city,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,104 Sushi & Co.,10422 168 St,Surrey,49.192059,-122.756256,0.0,Tea Room,Sushi Restaurant,Pizza Place,Coffee Shop,Women's Store,Fried Chicken Joint,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant
1,5 Star Catering,5640 188 St,Surrey,49.105449,-122.701315,0.0,Pub,Golf Course,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop
2,555 Pizza Ltd,9192 120 St,Surrey,49.170274,-122.889988,4.0,Thai Restaurant,Women's Store,Gift Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course
3,7-Eleven #26365,7986 120 St,Surrey,49.148189,-122.890221,0.0,American Restaurant,Steakhouse,Greek Restaurant,Gas Station,Other Repair Shop,Convenience Store,Coffee Shop,Indian Restaurant,Bus Stop,Fish & Chips Shop
4,7-Eleven #26517,17638 60 Ave,Surrey,49.111322,-122.734271,1.0,Gas Station,Convenience Store,Flea Market,Women's Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant


In [76]:
surrey_venues_sorted.head()

,Cluster Labels,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,104 Sushi & Co.,Tea Room,Sushi Restaurant,Pizza Place,Coffee Shop,Women's Store,Fried Chicken Joint,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant
1,0,5 Star Catering,Pub,Golf Course,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop
2,4,555 Pizza Ltd,Thai Restaurant,Women's Store,Gift Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course
3,0,7-Eleven #26365,American Restaurant,Steakhouse,Greek Restaurant,Gas Station,Other Repair Shop,Convenience Store,Coffee Shop,Indian Restaurant,Bus Stop,Fish & Chips Shop
4,1,7-Eleven #26517,Gas Station,Convenience Store,Flea Market,Women's Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant


### Cluster 1

##### This cluster is a cluster of yoga studios, stores, pizza and sandwich places, juice bars but mostly inclues the fish market nearby. What an useful insight !

In [78]:
surrey_merged.loc[surrey_merged['Cluster Labels'] == 0, surrey_merged.columns[[1] + list(range(5, surrey_merged.shape[1]))]]

# cluster1 = surrey_venues_sorted[surrey_venues_sorted['Cluster Labels'] == 0)]

,address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10422 168 St,0.0,Tea Room,Sushi Restaurant,Pizza Place,Coffee Shop,Women's Store,Fried Chicken Joint,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant
1,5640 188 St,0.0,Pub,Golf Course,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop
3,7986 120 St,0.0,American Restaurant,Steakhouse,Greek Restaurant,Gas Station,Other Repair Shop,Convenience Store,Coffee Shop,Indian Restaurant,Bus Stop,Fish & Chips Shop
7,15961 Fraser Hwy,0.0,Fast Food Restaurant,Gas Station,Pakistani Restaurant,Pizza Place,Convenience Store,Coffee Shop,Chinese Restaurant,Bank,Sushi Restaurant,Gourmet Shop
11,7195 King George Blvd,0.0,Gas Station,Pub,Sushi Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Women's Store
14,14445 64 Ave,0.0,Gas Station,Sandwich Place,Convenience Store,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Lebanese Restaurant,Gourmet Shop,Golf Course,Gift Shop
15,10376 152 St,0.0,Tea Room,Sushi Restaurant,Korean Restaurant,Coffee Shop,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant
17,6422 120 St,0.0,Coffee Shop,Pub,Convenience Store,Restaurant,Fast Food Restaurant,Gym / Fitness Center,Pizza Place,Supermarket,Gift Shop,Gas Station
18,10416 King George Blvd,0.0,Coffee Shop,Bank,Café,Gas Station,Sandwich Place,Women's Store,Gift Shop,Fish & Chips Shop,Flea Market,French Restaurant
19,15945 96 Ave,0.0,Indian Restaurant,Pizza Place,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Women's Store


In [79]:
surrey_merged.loc[surrey_merged['Cluster Labels'] == 1, surrey_merged.columns[[1] + list(range(5, surrey_merged.shape[1]))]]

,address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,17638 60 Ave,1.0,Gas Station,Convenience Store,Flea Market,Women's Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant
6,6021 168 St,1.0,Pizza Place,Convenience Store,Women's Store,Dim Sum Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station
8,13993 Fraser Hwy,1.0,Gas Station,Pharmacy,Convenience Store,Coffee Shop,Farmers Market,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Women's Store
9,6828 128 St,1.0,Health & Beauty Service,Convenience Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant
10,19188 72 Ave,1.0,Pizza Place,Convenience Store,Sandwich Place,Women's Store,Dim Sum Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint
12,19161 Fraser Hwy,1.0,Intersection,Convenience Store,Coffee Shop,Mexican Restaurant,Farmers Market,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station
13,9989 152 St,1.0,Chinese Restaurant,Convenience Store,Restaurant,Farmers Market,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Women's Store
37,9965 152 St,1.0,Chinese Restaurant,Convenience Store,Restaurant,Farmers Market,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Women's Store


In [80]:
surrey_merged.loc[surrey_merged['Cluster Labels'] == 2, surrey_merged.columns[[1] + list(range(5, surrey_merged.shape[1]))]]


,address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,6857 168 St,2.0,Golf Course,Women's Store,Dim Sum Restaurant,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station
21,6857 168 St,2.0,Golf Course,Women's Store,Dim Sum Restaurant,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station
22,6857 168 St,2.0,Golf Course,Women's Store,Dim Sum Restaurant,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station
23,6857 168 St,2.0,Golf Course,Women's Store,Dim Sum Restaurant,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station


In [81]:
surrey_merged.loc[surrey_merged['Cluster Labels'] == 3, surrey_merged.columns[[1] + list(range(5, surrey_merged.shape[1]))]]


,address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,14410 108 Ave,3.0,Convenience Store,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant
16,9583 132 St,3.0,Convenience Store,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant
29,13186 104 Ave,3.0,Convenience Store,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gift Shop,Dim Sum Restaurant


In [82]:
surrey_merged.loc[surrey_merged['Cluster Labels'] == 4, surrey_merged.columns[[1] + list(range(5, surrey_merged.shape[1]))]]


,address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,9192 120 St,4.0,Thai Restaurant,Women's Store,Gift Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course
39,7158 120 St,4.0,Thai Restaurant,Lounge,Women's Store,Gas Station,Fast Food Restaurant,Fish & Chips Shop,Flea Market,French Restaurant,Fried Chicken Joint,Golf Course


##### We can find the perfect spot for the new restaurant based on the above clusters